# Data Loading, EDA, Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.pipeline import Pipeline

import fancyimpute
import missingpy

%matplotlib inline

pd.options.display.max_columns = None

In [281]:
columns = [
    'age',
    'workclass',
    'final_weight', # this is number of people census thinks represented by this row
    'education',
    'education_num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hrs-per-week',
    'native-country',
    'income_class'
]
adult_df = pd.read_csv('data/adult.csv', delimiter=', ', names=columns)

In [282]:
adult_df['target'] = (adult_df['income_class']=='>50K').astype(int)

In [283]:
adult_df.describe()

,age,final_weight,education_num,capital_gain,capital_loss,hrs-per-week,target
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,0.240810
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,0.427581
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [284]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   final_weight    32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hrs-per-week    32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income_class    32561 non-null  object
 15  target          32561 non-null  int64 
dtypes: int64(7), object(9)
memory usage: 4.0+ MB


In [285]:
# Encode categorical variables
categorical_vars = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

#NOTE: OHE may be better here...idk
for feature in categorical_vars:
    encoder = LabelEncoder()
    adult_df[feature] = encoder.fit_transform(adult_df[feature])

In [286]:
adult_df.corr()['target']

age               0.234037
workclass         0.051604
final_weight     -0.009463
education         0.079317
education_num     0.335154
marital-status   -0.199307
occupation        0.075468
relationship     -0.250918
race              0.071846
sex               0.215980
capital_gain      0.223329
capital_loss      0.150526
hrs-per-week      0.229689
native-country    0.015840
target            1.000000
Name: target, dtype: float64

In [287]:
def drop_column_frac(df:pd.DataFrame, column_name:str, replace_with, drop_frac=0.1) -> pd.DataFrame:
    drop_idx = df[column_name].sample(frac=drop_frac).index
    dropped_df = df.copy()
    dropped_df.loc[drop_idx, column_name] = replace_with
    return dropped_df

In [288]:
def mean_imputation(df:pd.DataFrame) -> pd.DataFrame:
    return df.fillna(df.mean())

In [289]:
def drop_nas(df:pd.DataFrame) -> pd.DataFrame:
    return df.dropna()

In [290]:
def scale_features(train, test):
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.fit_transform(test)
    return train_scaled, test_scaled

In [291]:
def drop_columns(df):
    COLUMNS_TO_DROP = ['income_class', 'target']
    return df.drop(columns=COLUMNS_TO_DROP)

In [292]:
adult_df_full = adult_df.sample(frac=1) # shuffle data
adult_df = drop_column_frac(adult_df_full, 'education_num', np.nan, 0.4)
adult_df = drop_column_frac(adult_df, 'age', np.nan, 0.4)
adult_df = drop_column_frac(adult_df, 'sex', np.nan, 0.4)
adult_df = drop_column_frac(adult_df, 'capital_gain', np.nan, 0.4)
adult_df = drop_column_frac(adult_df, 'hrs-per-week', np.nan, 0.4)
mean_df = mean_imputation(adult_df)
dropped_df = drop_nas(adult_df)

target = adult_df['target']
target_dropped = dropped_df['target']
adult_df_full = drop_columns(adult_df_full)
mean_df = drop_columns(mean_df)
dropped_df = drop_columns(dropped_df)

In [293]:
class DataSetSplit:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

    def scaleFeatures(self):
        self.X_train, self.X_test = scale_features(self.X_train, self.X_test)


def split_data(df, target, test_size=0.2) -> DataSetSplit:
    X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=test_size)
    return DataSetSplit(X_train, X_test, y_train, y_test)

In [294]:
# split into test and training sets
full_data = split_data(adult_df_full, target)
mean_imputed = split_data(mean_df, target)
dropped_data = split_data(dropped_df, target_dropped)

In [295]:
# Scale features
full_data.scaleFeatures()
mean_imputed.scaleFeatures()
dropped_data.scaleFeatures()

data_sets = { 
    'full': full_data,
    'mean': mean_imputed,
    'drop': dropped_data
}

In [307]:
for name, data in data_sets.items():
    log_reg = LogisticRegression()
    log_reg.fit(data.X_train, data.y_train)
    y_predicted = log_reg.predict(data.X_test)
    print(f'Logistic regression {name} Accuracy: {accuracy_score(y_pred=y_predicted, y_true=data.y_test)}')


Logistic regression full Accuracy: 0.826654383540611
Logistic regression mean Accuracy: 0.8050053738676494
Logistic regression drop Accuracy: 0.8287937743190662


In [ ]:
# Start exploring imputing with neural networks

In [303]:
data_sets.items()

dict_items([('full', <__main__.DataSetSplit object at 0x7fd38b7477f0>), ('mean', <__main__.DataSetSplit object at 0x7fd34cd042e0>), ('drop', <__main__.DataSetSplit object at 0x7fd3a35cc400>)])

# Zillow Data

In [3]:
# Using training data set since the properties datasets are HUGE
zillow_df_original = pd.read_csv('data/zillow-prize-1/properties_2017_smaller.csv')
zillow_df_dummy = pd.get_dummies(zillow_df)

In [4]:
print(zillow_df_original.shape)
print(zillow_df_dummy.shape)
zillow_category_df = zillow_df_original.select_dtypes('object')
zillow_dummy_df_objects = pd.get_dummies(zillow_category_df)
print(zillow_dummy_df_objects.shape)

(298522, 59)
(298522, 3170)
(298522, 3116)


In [31]:
# # One hot encode the variables
# zillow_category_df = zillow_df.select_dtypes('object')
# zillow_dummy_df = pd.get_dummies(zillow_category_df)
# zillow_dummy_df.head()

0   

   propertyzoningdesc_WCRA  propertyzoningdesc_WCRA*  \
0                        0                         0   
1                        0                         0   
2                        0                         0   
3                        0                         0   
4                        0                         0   

   propertyzoningdesc_WCRA20000*  propertyzoningdesc_WCRA40000*  \
0                              0                              0   
1                              0                              0   
2                              0                              0   
3                              0                              0   
4                              0                              0   

   propertyzoningdesc_WCRAYY  propertyzoningdesc_WD4*  \
0                          0                        0   
1                          0                        0   
2                          0                        0   
3                          0                        0   
4                          0                        0   

   propertyzoningdesc_WD4R4*  propertyzoningdesc_WDC1A*  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   propertyzoningdesc_WDC1B*  propertyzoningdesc_WDC2A*  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   propertyzoningdesc_WDC3A*  propertyzoningdesc_WDR1A*  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   propertyzoningdesc_WDR1B*  propertyzoningdesc_WDR1B+PK*  \
0                          0                             0   
1                          0                             0   
2                          0                             0   
3                          0                             0   
4                          0                             0   

   propertyzoningdesc_WDR2*  propertyzoningdesc_WDR3A*  \
0                         0                          0   
1                         0                          0   
2                         0                          0   
3                         0                          0   
4                         0                          0   

   propertyzoningdesc_WDR3B*  propertyzoningdesc_WDR3C*  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   propertyzoningdesc_WDR4*  propertyzoningdesc_WDR4+PK*  \
0                         0                            0   
1                         0                            0   
2                         0                            0   
3                         0                            0   
4                         0                            0   

   propertyzoningdesc_WH R1*  propertyzoningdesc_WHC0*  \
0                          0                         0   
1                          0                         0   
2                          0                         0   
3                          0                         0   
4                          0                         0   

   propertyzoningdesc_WHC0UD*  propertyzoningdesc_WHC1*  \


In [33]:
zillow_df.corr()[TRUE_TARGET].sort_values(ascending=False)

In [14]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298522 entries, 0 to 298521
Data columns (total 59 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    298522 non-null  int64  
 1   parcelid                      298522 non-null  int64  
 2   airconditioningtypeid         81386 non-null   float64
 3   architecturalstyletypeid      608 non-null     float64
 4   basementsqft                  170 non-null     float64
 5   bathroomcnt                   298225 non-null  float64
 6   bedroomcnt                    298226 non-null  float64
 7   buildingclasstypeid           1228 non-null    float64
 8   buildingqualitytypeid         194234 non-null  float64
 9   calculatedbathnbr             286773 non-null  float64
 10  decktypeid                    1721 non-null    float64
 11  finishedfloor1squarefeet      20262 non-null   float64
 12  calculatedfinishedsquarefeet  294042 non-nul

In [24]:
TRUE_TARGET = 'taxvaluedollarcnt'
PERFECT_COR = ['taxamount', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', TRUE_TARGET]
CORR_THRESHOLD = 0.05

# Only keep colms that have correlation coeff greater than CORR_THRESHOLD
correlation_array = zillow_df.corr()[TRUE_TARGET]
colms_to_keep = correlation_array[correlation_array > CORR_THRESHOLD]
parsed_zillow_df = zillow_df[colms_to_keep.index]

# Extract target
target = parsed_zillow_df[TRUE_TARGET]

# Drop colmns that corr very strongly with tax dollar cnt
parsed_zillow_df = parsed_zillow_df.drop(columns=PERFECT_COR)

In [26]:
parsed_zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298522 entries, 0 to 298521
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   basementsqft                  170 non-null     float64
 1   bathroomcnt                   298225 non-null  float64
 2   bedroomcnt                    298226 non-null  float64
 3   buildingqualitytypeid         194234 non-null  float64
 4   calculatedbathnbr             286773 non-null  float64
 5   finishedfloor1squarefeet      20262 non-null   float64
 6   calculatedfinishedsquarefeet  294042 non-null  float64
 7   finishedsquarefeet12          272160 non-null  float64
 8   finishedsquarefeet13          797 non-null     float64
 9   finishedsquarefeet15          18886 non-null   float64
 10  finishedsquarefeet50          20262 non-null   float64
 11  finishedsquarefeet6           2199 non-null    float64
 12  fireplacecnt                  31251 non-null

In [25]:
parsed_zillow_df.head()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,poolsizesum,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories
0,NaN,3.0,3.0,8.0,3.0,NaN,1231.0,1231.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1980.0,NaN
1,NaN,4.0,6.0,NaN,4.0,NaN,3682.0,NaN,NaN,3682.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1919.0,NaN
2,NaN,3.0,3.0,7.0,3.0,NaN,1570.0,1570.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2006.0,NaN
3,NaN,3.0,3.0,8.0,3.0,NaN,2518.0,2518.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1958.0,NaN
4,NaN,4.0,5.0,5.0,4.0,NaN,3298.0,NaN,NaN,3298.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1945.0,NaN


# =================
# ===================
# ===================

In [3]:
# Using training data set since the properties datasets are HUGE
zillow_df = pd.read_csv('data/zillow-prize-1/properties_2017_smaller.csv')

# Number of highest correlation values to keep
NUM_COLS = 10

cols_to_keep = zillow_df.corr()['taxvaluedollarcnt'].sort_values(ascending=False)[0:10]

parsed_zillow_df = zillow_df[list(cols_to_keep.index)]

# Drop rows where true target is 0
TRUE_TARGET = 'taxvaluedollarcnt'
parsed_zillow_df = parsed_zillow_df[parsed_zillow_df[TRUE_TARGET].notna()]

# Can adjust the columns here, but mostly just care about imputation

In [4]:
target = parsed_zillow_df[TRUE_TARGET]

# Drop colmns that corr very strongly with tax dollar cnt
PERFECT_COR = ['taxamount', 'structuretaxvaluedollarcnt', TRUE_TARGET]
parsed_zillow_df = parsed_zillow_df.drop(columns=PERFECT_COR)

# Perform mean imputation

In [5]:
# Perform mean imputations on each colm with missing data
mean_imputed_df = parsed_zillow_df.fillna(parsed_zillow_df.mean())

# MICE Imputation

In [6]:
temp = fancyimpute.IterativeImputer().fit_transform(parsed_zillow_df)
mice_imputed_df = pd.DataFrame(data=temp, index=parsed_zillow_df.index, columns=parsed_zillow_df.columns)

# Perform KNN Imputation

## Leaving this error here b/c it's interesting....this has LOTS of missing data, if had less missing data then different approaches maybe needed

In [7]:
tempknn = missingpy.KNNImputer().fit_transform(parsed_zillow_df.to_numpy())
knn_imputed_df = pd.DataFrame(data=tempknn, index=parsed_zillow_df.index, columns=parsed_zillow_df.columns)

ValueError: Some column(s) have more than 80.0% missing values

# Random Forest Imputation

In [8]:
temp_rf = missingpy.MissForest().fit_transform(parsed_zillow_df.to_numpy())
rf_imputed_df = pd.DataFrame(data=temp_rf, index=parsed_zillow_df.index, columns=parsed_zillow_df.columns)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [9]:
# Returns best_score_, best_params_, grid_search object
def grid_search(X:np.array, y:np.array, pipeline: Pipeline, params_grid: dict, n_jobs=4) -> (float, dict, float):
    search = GridSearchCV(pipeline, params_grid, n_jobs=n_jobs)
    search.fit(X, y)
    return search.best_score_, search.best_params_, search

In [10]:
def run_pipeline(X_train:np.array, y_train:np.array, X_test:np.array, y_test:np.array, pipeline: Pipeline) -> float:
    pipeline.fit(X_train, y_train)
    return pipeline.score(X_test, y_test)

In [11]:
# Train test split
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(mean_imputed_df.to_numpy(), target, test_size=0.2)
X_train_mice, X_test_mice, y_train_mice, y_test_mice = train_test_split(mice_imputed_df.to_numpy(), target, test_size=0.2)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(rf_imputed_df.to_numpy(), target, test_size=0.2)

In [12]:
poly_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lin_reg', linear_model.LinearRegression())
])

# Train optimal degree
mean_reg_score = run_pipeline(X_train_mean, y_train_mean, X_test_mean, y_test_mean, poly_pipeline)
print('Poly testing score for mean imputed: %0.4f' % mean_reg_score)

mice_reg_score = run_pipeline(X_train_mice, y_train_mice, X_test_mice, y_test_mice, poly_pipeline)
print('Poly testing score for MICE imputed: %0.4f' % mice_reg_score)

rf_reg_score = run_pipeline(X_train_rf, y_train_rf, X_test_rf, y_test_rf, poly_pipeline)
print('Poly testing score for random forest imputed: %0.4f' % rf_reg_score)

Poly testing score for mean imputed: -0.4999
Poly testing score for MICE imputed: 0.9258
Poly testing score for random forest imputed: 0.9091


# Below is basically trying to find the best model to use as our metric sort of...although maybe there are better approaches to determining if our imputation has "done well"

In [17]:
linear_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('lin_reg', linear_model.LinearRegression())
])

lin_reg_score = run_pipeline(X_train_mean, y_train_mean, X_test_mean, y_test_mean, linear_pipeline)
print('Linear regression score: %0.4f' % lin_reg_score)

Linear regression score: 0.8214


# TODO GridSearch for best degree?

In [18]:
poly_param_grid = {
    'poly_features__degree': [2,3,4,5]
}

poly_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lin_reg', linear_model.LinearRegression())
])

poly_grid_score, poly_grid_params, poly_search = grid_search(X_train_mean, y_train_mean, poly_pipeline, poly_param_grid)
print('Best parameters: (CV score=%0.3f:' % search.best_score_)
print(poly_grid_params)

KeyboardInterrupt: 

In [58]:
poly_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lin_reg', linear_model.LinearRegression())
])

# Train optimal degree
poly_reg_score = run_pipeline(X_train_mean, y_train_mean, X_test_mean, y_test_mean, poly_pipeline)
print('Poly testing score: %0.4f' % poly_reg_score)

Poly testing score: 0.93


# Lasso Regression

In [61]:
lasso_param_grid = {
    'poly_features__degree': [1, 2, 3, 4, 5],
    'lasso_reg__alpha': [0.1, 0.5, 1.0, 1.5]
}

# Lasso pipeline
lasso_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=1)),
    ('lasso_reg', linear_model.Lasso(alpha=0.5))
])

lasso_grid_score, lasso_grid_params, lasso_search = grid_search(X_train_mean, y_train_mean, lasso_pipeline, lasso_param_grid)
print('Best parameters: (CV score=%0.3f:' %lasso_grid_score)
print(lasso_grid_params)

Best parameters: (CV score=0.910:
{'lasso_reg__alpha': 1.5, 'poly_features__degree': 1}


In [63]:
optimal_lasso_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=1)),
    ('lasso_reg', linear_model.Lasso(alpha=1.5))
])

lasso_pipeline.fit(X_train_mean, y_train_mean)
print('Lasso testing score: %0.4f' % lasso_pipeline.score(X_test_mean,y_test_mean))

Lasso testing score: 0.8989


# Ridge Regression

In [19]:
ridge_param_grid = {
    'poly_features__degree': [1, 2, 3, 4],
    'ridge_reg__alpha': [0.001, 0.1, 0.5, 1.0, 1.5]
}

# Ridge pipeline
ridge_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=1)),
    ('ridge_reg', linear_model.Ridge(alpha=0.5))
])

poly_grid_score, poly_grid_params, poly_search = grid_search(X_train_mean, y_train_mean, ridge_pipeline, ridge_param_grid)
print('Best parameters: (CV score=%0.3f:' % poly_grid_score)
print(poly_grid_params)

Best parameters: (CV score=0.794:
{'poly_features__degree': 1, 'ridge_reg__alpha': 1.5}


In [20]:
optimal_ridge_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=1)),
    ('lasso_reg', linear_model.Ridge(alpha=0.001))
])

optimal_ridge_pipeline.fit(X_train_mean, y_train_mean)
print('Poly testing score: %0.4f' % optimal_ridge_pipeline.score(X_test_mean,y_test_mean))

Poly testing score: 0.8214


# Explore various imputation methods w/ fancyimpute

In [81]:
knn_zillow_imputed = fancyimpute.KNN(k=3).fit_transform(parsed_zillow_df)

In [22]:
knn_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('knn_impute', fancyimpute.KNN(k=3)),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lin_reg', linear_model.LinearRegression())
])

knn_pipeline.fit(X_train, y_train)
print('Testing score: %0.4f' % knn_pipeline.score(X_test, y_test))